# Scalar Bhattacharyya Distance

In [20]:
import numpy as np
import pandas as pd
from sklearn.covariance import LedoitWolf
from scipy.linalg import det, inv, cho_factor, cho_solve

genes = pd.read_csv("C:/Users/Brayan Gutierrez/Desktop/RNAseq-AMD/Dataset/aak100_cpmdat.csv")

control = genes[genes['mgs_level'] == "MGS1"]
late = genes[genes['mgs_level'] == "MGS4"]

In [21]:
control_genes = control.drop(["Unnamed: 0", 'mgs_level'], axis=1)
late_genes = late.drop(["Unnamed: 0", 'mgs_level'], axis=1)

In [30]:
control_mean = np.mean(control_genes, axis=0)
late_mean = np.mean(late_genes, axis=0)

control_cov = np.cov(control_genes.T)
late_cov = np.cov(late_genes.T)

pooled_cov = 0.5 * (late_cov + control_cov)
delta = control_mean - late_mean

L, lower = cho_factor(pooled_cov, lower=True)

x = cho_solve((L, lower), delta)

quad_term = 0.125 * delta.T @ x

In [31]:
# Symmetrize just to be safe
Sigma  = 0.5 * (pooled_cov  + pooled_cov.T)
Sigma1 = 0.5 * (control_cov + control_cov.T)
Sigma2 = 0.5 * (late_cov + late_cov.T)

# --- Compute log-determinants via Cholesky ---
def logdet_cholesky(M, eps=1e-6, max_iter=5):
    """Compute log(det(M)) safely using Cholesky, adding ridge if needed."""
    for i in range(max_iter):
        try:
            L, lower_flag = cho_factor(M, lower=True)
            return 2.0 * np.sum(np.log(np.diag(L)))
        except np.linalg.LinAlgError:
            # Add small ridge and retry
            M = M + eps * np.eye(M.shape[0])
            eps *= 10  # increase ridge if it still fails
    raise np.linalg.LinAlgError("Matrix is not positive definite even after regularization.")

logdet_Sigma  = logdet_cholesky(Sigma)
logdet_Sigma1 = logdet_cholesky(Sigma1)
logdet_Sigma2 = logdet_cholesky(Sigma2)

# --- Combine the terms for the Bhattacharyya log-det component ---
logdet_term = 0.5 * (logdet_Sigma - 0.5 * (logdet_Sigma1 + logdet_Sigma2))

In [33]:
results = quad_term + logdet_term

results

78.08849138763478

# Bhattacharyya Distance per Gene

In [34]:
import numpy as np
import pandas as pd

# Load data
genes = pd.read_csv("C:/Users/Brayan Gutierrez/Desktop/RNAseq-AMD/Dataset/aak100_cpmdat.csv")

# Split into groups
control = genes[genes['mgs_level'] == "MGS1"]
late = genes[genes['mgs_level'] == "MGS4"]

# Keep only numeric gene expression columns
control_genes = control.drop(["Unnamed: 0", "mgs_level"], axis=1)
late_genes = late.drop(["Unnamed: 0", "mgs_level"], axis=1)

# Compute means and standard deviations per gene
mu1 = np.mean(control_genes, axis=0)
mu2 = np.mean(late_genes, axis=0)
sigma1 = np.std(control_genes, axis=0, ddof=1)
sigma2 = np.std(late_genes, axis=0, ddof=1)

# --- Univariate Bhattacharyya distance for each gene ---
# Formula:
# D_B = 1/4 * ((μ1 - μ2)^2 / (σ1^2 + σ2^2)) + 1/2 * ln( (σ1^2 + σ2^2) / (2σ1σ2) )

term1 = 0.25 * ((mu1 - mu2)**2) / (sigma1**2 + sigma2**2)
term2 = 0.5 * np.log((sigma1**2 + sigma2**2) / (2 * sigma1 * sigma2))
D_B = term1 + term2

# Store results in a DataFrame
bhattacharyya_df = pd.DataFrame({
    "Gene": control_genes.columns,
    "Bhattacharyya_Distance": D_B
})

# Save to CSV for use in MEGENA (R)
bhattacharyya_df.to_csv("C:/Users/Brayan Gutierrez/Desktop/RNAseq-AMD/Dataset/bhattacharyya_per_gene.csv", index=False)

print("✅ Bhattacharyya distances computed per gene and saved to CSV.")

✅ Bhattacharyya distances computed per gene and saved to CSV.


# Bhattacharyya Distance Edges

## Control + Late

In [1]:
import numpy as np
import pandas as pd

genes = pd.read_csv("C:/Users/Brayan Gutierrez/Desktop/RNAseq-AMD/Dataset/aak100_cpmdat.csv")

# Keep only expression columns
expr = genes.drop(["Unnamed: 0", "mgs_level"], axis=1)
gene_names = expr.columns
expr_values = expr.to_numpy()

edges = []

# Compute pairwise Bhattacharyya distances between genes
for i in range(len(gene_names)):
    for j in range(i+1, len(gene_names)):
        g1 = expr_values[:, i]
        g2 = expr_values[:, j]

        # Compute mean and std per gene (overall or per condition)
        mu1, mu2 = np.mean(g1), np.mean(g2)
        sigma1, sigma2 = np.std(g1, ddof=1), np.std(g2, ddof=1)

        # Univariate Bhattacharyya distance
        term1 = 0.25 * ((mu1 - mu2)**2) / (sigma1**2 + sigma2**2)
        term2 = 0.5 * np.log((sigma1**2 + sigma2**2) / (2 * sigma1 * sigma2))
        D_B = term1 + term2

        edges.append([gene_names[i], gene_names[j], D_B])

# Create dataframe in MEGENA format
edges_df = pd.DataFrame(edges, columns=["from", "to", "distance"])

edges_df.to_csv("C:/Users/Brayan Gutierrez/Desktop/RNAseq-AMD/Dataset/bhattacharyya_edges.csv", index=False)


## Control

## Late